In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/citation_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/citation_sum


In [ ]:
!pip3 install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 5.0 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.8 MB/s 
     |████████████████████████████████| 895 kB 45.9 MB/s 


In [ ]:
!pip3 install torch
!pip3 install sentencepiece

     |████████████████████████████████| 1.2 MB 5.0 MB/s 


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

import torch
import torch.nn.functional as F

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from collections import defaultdict

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

## Load the fine-tuned baseline (50k on arXiv dataset) or GPT-2 model
## Citation contexts with their respective reference article's abstract----this is just for one field of study

In [ ]:
model = BartForConditionalGeneration.from_pretrained("../citation_sum/BART-checkpoints/")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

In [ ]:
model = model.to(device)

In [ ]:
# Generate 5 abstractive citation contexts for each citing sentence passed here
def _generate_abstractive_citation_context(citing_sentence, num_return_sequences=5):
  # Generate Summary

  inputs = tokenizer(citing_sentence, truncation=True, padding='longest',return_tensors='pt').to(device)
  summary_ids = model.generate(inputs['input_ids'], num_beams=5, min_length=20, 
                               max_length=50, early_stopping=True, num_return_sequences=num_return_sequences)

  lst_tgt_text = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)   # This is a list of abstractive citation contexts
  return lst_tgt_text


## Iterate through the 1000 reference papers in the scisummnet corpus and call the method defined above

In [ ]:
def main():
  DATA_PATH = "ScisummNet/scisummnet_release1.1__20190413/top1000_complete"
  w_rp_abstract = True

  if w_rp_abstract:
    OUTPUT_PATH = "abstractive_citation_contexts_SciSummNet/ABSTRACTIVE_CITATION_CONTEXTS_FROM_CITATIONS_AND_RP_ABSTRACT"
  else:
    OUTPUT_PATH = "abstractive_citation_contexts_SciSummNet/ABSTRACTIVE_CITATION_CONTEXTS_FROM_CITATIONS_ONLY"

  os.makedirs(f"{OUTPUT_PATH}", exist_ok=True)

  for paper_id in os.listdir(DATA_PATH):
    abstractive_citation_contexts = list()
    citing_sentences = list()   # to store all incoming citing sentences
    for file in os.listdir(os.path.join(DATA_PATH, paper_id)):
      if file.endswith('.json'):
        with open(os.path.join(f"{DATA_PATH}/{paper_id}", file), 'r') as fp:
          data = json.load(fp)
        fp.close()
        citing_sentences = [obj['clean_text'] for obj in data]  # citing sentences pertaining to a cited paper id

    # read reference paper abstract
    with open(f"{DATA_PATH}/{paper_id}/referece_paper_abstract.txt", 'r') as fp:
      rp_abstract = fp.read().strip()
    fp.close()

    for citing_sent in citing_sentences:
      # call to the method generating the abstractive summaries
      if w_rp_abstract:
        input_to_generative_model = rp_abstract + " " + citing_sent
        lst_abstsractive_citation_contexts = _generate_abstractive_citation_context(input_to_generative_model)  # for all abstractive citation contexts
      else:
        input_to_generative_model = citing_sent
        lst_abstsractive_citation_contexts = _generate_abstractive_citation_context(input_to_generative_model)  # for all abstractive citation contexts
      
      abstractive_citation_contexts += lst_abstsractive_citation_contexts
    
    with open(f"{OUTPUT_PATH}/{paper_id}.txt", 'w') as fp:
      for acc in abstractive_citation_contexts:
        fp.write(str(acc))
        fp.write('\n')
    fp.close()


In [ ]:
if __name__ == "__main__":
  main()

## Read back one of the files (note that file name is cited paper id) containing the abstractive citation contexts to verify

In [ ]:
OUTPUT_PATH = "abstractive_citation_contexts_SciSummNet/ABSTRACTIVE_CITATION_CONTEXTS_FROM_CITATIONS_AND_RP_ABSTRACT"
paper_id = 'A00-2024'

with open(f"{OUTPUT_PATH}/{paper_id}.txt", 'r') as fp:
  accs = fp.read().splitlines()
fp.close()